# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [42]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import openweathermapy.core as owm
import urllib
from datetime import datetime
from pprint import pprint

# Import API key
from api_keys import api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [43]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

606

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [ ]:
settings = {"units": "imperial", "appid": api_key}
summary = []

print("Beginning Data Retrieval\n-----------------------------")
for city in cities:
    try:
        views = ["name", "clouds.all", "sys.country", "dt", "main.humidity", \
                  "coord.lat", "coord.lon", "main.temp_max", "main.pressure", "wind.speed"]
        data = owm.get_current(city, **settings).get_dict(views, split_keys=True)
        summary.append( [data['name'], data['all'], data['country'],data['dt'], data['humidity'], \
                 data['lat'], data['lon'], data['pressure'], data['temp_max'], data['speed'] ] )
        print(f'Processing Record 1 of Set 1 | {city}')
    except KeyError as e:
        print(f'{city} not found with error:{e}. Skipping ...')
    except urllib.error.HTTPError as e:
        print(f'{city} not found with error:{e}  Skipping ...')

Beginning Data Retrieval
-----------------------------
Processing Record 1 of Set 1 | alofi
Processing Record 1 of Set 1 | busselton
Processing Record 1 of Set 1 | rikitea
Processing Record 1 of Set 1 | hermanus
Processing Record 1 of Set 1 | barrow
Processing Record 1 of Set 1 | amuntai
Processing Record 1 of Set 1 | nadym
Processing Record 1 of Set 1 | kumeny
Processing Record 1 of Set 1 | hobart
Processing Record 1 of Set 1 | katsuura
Processing Record 1 of Set 1 | dikson
Processing Record 1 of Set 1 | omboue
Processing Record 1 of Set 1 | ushuaia
Processing Record 1 of Set 1 | ostrovnoy
Processing Record 1 of Set 1 | bluff
chikoy not found with error:HTTP Error 404: Not Found  Skipping ...
Processing Record 1 of Set 1 | victoria
Processing Record 1 of Set 1 | ancud
Processing Record 1 of Set 1 | mataura
Processing Record 1 of Set 1 | kavaratti
Processing Record 1 of Set 1 | saskylakh
Processing Record 1 of Set 1 | cayenne
Processing Record 1 of Set 1 | hilo
Processing Record 1 of S

Processing Record 1 of Set 1 | saint anthony
Processing Record 1 of Set 1 | norman wells
olafsvik not found with error:HTTP Error 404: Not Found  Skipping ...
Processing Record 1 of Set 1 | yuksekova
Processing Record 1 of Set 1 | saldanha
Processing Record 1 of Set 1 | canutama
Processing Record 1 of Set 1 | taywarah
Processing Record 1 of Set 1 | grand island
Processing Record 1 of Set 1 | preston
Processing Record 1 of Set 1 | xunchang
Processing Record 1 of Set 1 | tuyen quang
Processing Record 1 of Set 1 | meadow lake
Processing Record 1 of Set 1 | te anau
Processing Record 1 of Set 1 | banepa
Processing Record 1 of Set 1 | atherton
Processing Record 1 of Set 1 | varhaug
Processing Record 1 of Set 1 | jacmel
Processing Record 1 of Set 1 | scottsbluff
Processing Record 1 of Set 1 | hasaki
Processing Record 1 of Set 1 | menongue
Processing Record 1 of Set 1 | laguna
rawannawi not found with error:HTTP Error 404: Not Found  Skipping ...
Processing Record 1 of Set 1 | upernavik
Proces

Processing Record 1 of Set 1 | kaitangata
Processing Record 1 of Set 1 | greece
Processing Record 1 of Set 1 | murindo
Processing Record 1 of Set 1 | hamilton
Processing Record 1 of Set 1 | port hardy
Processing Record 1 of Set 1 | port lincoln
Processing Record 1 of Set 1 | mentougou
Processing Record 1 of Set 1 | kirakira
Processing Record 1 of Set 1 | tumwater
Processing Record 1 of Set 1 | srednekolymsk
Processing Record 1 of Set 1 | naze
Processing Record 1 of Set 1 | honningsvag
Processing Record 1 of Set 1 | burlington
Processing Record 1 of Set 1 | tautira
Processing Record 1 of Set 1 | shelburne
Processing Record 1 of Set 1 | mae sai
Processing Record 1 of Set 1 | millau
Processing Record 1 of Set 1 | warrnambool
Processing Record 1 of Set 1 | hollins
Processing Record 1 of Set 1 | xuddur
Processing Record 1 of Set 1 | kosh-agach
Processing Record 1 of Set 1 | sao joao da barra
Processing Record 1 of Set 1 | sioux lookout
Processing Record 1 of Set 1 | castanos
Processing Reco

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [ ]:
weather_df = pd.DataFrame(summary)
weather_df.columns = ["name", "Cloudiness", "Country", "Date", "Humidity", \
                  "Lat", "Lng", "Pressure", "Max Temp", "Wind Speed"]

weather_df.to_csv(output_data_file,
                  encoding="utf-8", index=False, header=True)
weather_df.count()

In [ ]:
weather_df.head(5)

In [ ]:
# A general function to plat all weather related scatter diagram
def weather_plot(h_title, y_label, what_plot, y_lim, mark_adj):
    """
    Args:
          h_title    - title string in the top of plot
          y_label    - Vertical label in the left side of plot
          what_plot  - The attribute to plot
          y_lim      - A pair of int in a list to reconfig y range
          mark_adj   - integer to rescale the marker size
    Return:
        none
    """
    ts =  int(max([ i[3] for i in summary ]))
    yymmdd = '(' + str(datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S')[0:10].replace('-','/')) + ')'
    figure_title="City Latitude vs. " + h_title + " Max Temperature " + yymmdd
    fig = weather_df.plot(kind="scatter", x="Lat", y=what_plot, grid=True, figsize=(6,4), marker="o",
            facecolors="SkyBlue", edgecolors="black", s=sum(weather_df[what_plot]) / mark_adj)
    plt.title(figure_title, fontsize = 12, horizontalalignment='center')
    fig.set_ylabel(y_label, fontsize = 12)
    fig.set_xlabel("Latitude", fontsize = 12)
    y0, y1 = fig.get_ylim()
    fig.set_ylim(y0 + y_lim[0] , y1 + y_lim[1])
    plt.show()

### Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

#### Latitude vs. Temperature Plot

In [ ]:
weather_plot('Max Temperature', 'Max Temperature(F)', 'Max Temp', [0, 0], 500) 

#### Latitude vs. Humidity Plot

In [ ]:
weather_plot('Humidity', 'Humidity (%)', 'Humidity', [0, 0], 600) 

#### Latitude vs. Cloudiness Plot

In [ ]:
weather_plot('Cloudiness', 'Cloudiness (%)', 'Cloudiness', [0, 0], 400) 

#### Latitude vs. Wind Speed Plot

In [ ]:
weather_plot('Wind Speed', 'Wind Speed (mph)', 'Wind Speed', [1, -1], 60) 